# Multi-scale gradient / Unet GAN 8x8-as változata (egy kapcsolattal)

// Ez csak egy jegyzet és kezdetleges verzió //
A 4x4-es egy szintel való kibővítése,
a generátor kigenerálja a 4x4-es és 8x8-as képeket, amelyeket átad a diszkriminátornak.
A kapcsolat a 4x4-es képnél van (Concat művelet)

a tanulás során a 8x8-as kimeneten hamarabb konvergál a megoldás felé a generátor
Ezt megfigyeltem a nagyobb modellnél is, hogy a nagyobb felbontáson tanulja be a mintákat először, majd a kisseb felbontásokon...

16x16-nál egyből mode collapse lép fel már így haladva...

Lehetséges, hogy a loss function-ön kell valamit módosítani (ez nem volt teljesen egyértelmű az MSG cikkben)

In [1]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import AveragePooling2D
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import Concatenate
import matplotlib.ticker as ticker

from tensorflow import random
from tensorflow import GradientTape

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import time

from IPython import display

In [2]:
epochs = 50
number_of_examples = 8
batch_size = 16
latent_dim = 512
image_size = (8, 8) # h x w

seed = tf.random.normal([number_of_examples, latent_dim])

data_dir = r'../input/animal-faces/afhq/train/'

dataset = image_dataset_from_directory(
    data_dir, label_mode=None, image_size=image_size, batch_size=batch_size
)

In [3]:
dataset = dataset.map(lambda x: (x - 127.5) / 127.5) # Normalizing to -1,1

In [4]:
def make_generator_model(latent_dim):
    visible = Input(shape=[latent_dim])
    hidden = Reshape((1, 1, latent_dim))(visible)

    hidden = Conv2DTranspose(filters=512, kernel_size=4, strides=(1, 1),
                             padding='valid', activation="leaky_relu")(hidden)
    hidden = Conv2D(filters=512, kernel_size=3, strides=(1, 1),
                             padding='same', activation="leaky_relu")(hidden)
    
    hidden = BatchNormalization()(hidden)
    out4x4 = hidden
    out4x4 = Conv2D(filters=3, kernel_size=4, strides=(1, 1), padding='same')(out4x4)
    out4x4 = Activation("tanh")(out4x4)
    
    
    hidden = UpSampling2D(interpolation="bilinear")(hidden)

    
    hidden = Conv2D(filters=256, kernel_size=3, strides=(1, 1),
                             padding='same', activation="leaky_relu")(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Conv2D(filters=256, kernel_size=3, strides=(1, 1),
                             padding='same', activation="leaky_relu")(hidden)
    hidden = BatchNormalization()(hidden)
    out8x8 = hidden
    out8x8 = Conv2D(filters=3, kernel_size=4, strides=(1, 1), padding='same')(out8x8)
    out8x8 = Activation("tanh")(out8x8)
    

    model = Model(inputs=visible, outputs=[out8x8, out4x4])
    
    return model

In [5]:
generator = make_generator_model(latent_dim)

noise = tf.random.normal([1, latent_dim])
generated_image = generator(noise, training=False)

print(generated_image[0].shape)

plt.imshow((generated_image[0][0].numpy()*127.5+127.5).astype("uint32"))

In [6]:
generator.summary()

In [7]:
def make_discriminator_model():
    input4 = Input(shape=(8, 8, 3))
    
    hidden = Conv2D(filters=256, kernel_size=4, strides=1, padding="same",
                    activation="tanh")(input4) # From RGB

    hidden = Conv2D(filters=256, kernel_size=3, strides=1, padding="same",
                    activation="leaky_relu")(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Conv2D(filters=512, kernel_size=3, strides=1, padding="same",
                    activation="leaky_relu")(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = AveragePooling2D()(hidden)
    
    
    input5 = Input(shape=(4, 4, 3))
    hidden = Concatenate()([input5, hidden])
    
    hidden = Conv2D(filters=512, kernel_size=3, strides=1, padding="same",
                    activation="leaky_relu")(hidden)
    hidden = BatchNormalization()(hidden)
    
    
    hidden = Conv2D(filters=512, kernel_size=4, strides=1, padding="valid", activation="leaky_relu")(hidden)
    hidden = Flatten()(hidden)
    hidden = Dense(1)(hidden)
    out = Activation("sigmoid")(hidden)

    model = Model(inputs=[input4, input5], outputs=[out])
    
    return model

In [8]:
discriminator = make_discriminator_model()
discriminator.summary()

In [9]:
plot_model(discriminator)

In [10]:
decision = discriminator(generated_image, training=False)
print(decision)

In [11]:
if not os.path.isdir("epochs"):
    os.mkdir("epochs")
def plot_examples(images_at_scales, epoch):
    examples = images_at_scales[0].shape[0]
    output_lenght = len(images_at_scales)

    fig, axes = plt.subplots(figsize=(output_lenght, examples),
                             nrows=examples, ncols=output_lenght, dpi=200)

    for i in range(examples):
        for j in range(output_lenght):
            generated_images = images_at_scales[output_lenght-1-j]
            gen_1 = generated_images[i].numpy()*127.5+127.5
            axes[i, j].axis('off')
            axes[i, j].imshow((gen_1).astype("uint32"), interpolation='none')
    plt.show()

In [12]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    total_loss = real_loss + fake_loss
    return total_loss

# The generator is performing well, if the discriminator classifies fakes as real(1)
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [13]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
#generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)

discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
#discriminator_optimizer = tf.keras.optimizers.RMSprop(1e-4)

In [14]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [15]:
@tf.function
def train_step(images):
    noise = random.normal([batch_size, latent_dim])

    with GradientTape() as gen_tape, GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(
        gen_loss,
        generator.trainable_variables
    )

    gradients_of_discriminator = disc_tape.gradient(
        disc_loss,
        discriminator.trainable_variables
    )

    generator_optimizer.apply_gradients(
        zip(gradients_of_generator,
            generator.trainable_variables)
        )
    discriminator_optimizer.apply_gradients(
        zip(gradients_of_discriminator,
            discriminator.trainable_variables)
        )

    return (gen_loss, disc_loss)

In [16]:
def scale_input_images(image_batch):
    result = []
    result.append(image_batch)
    #result.append(tf.image.resize(image_batch, [32, 32]))
    #result.append(tf.image.resize(image_batch, [16, 16]))
    #result.append(tf.image.resize(image_batch, [8, 8]))
    result.append(tf.image.resize(image_batch, [4, 4]))
    
    return result

In [17]:
def train(dataset, epochs):
    generator_losses = np.empty((0, 0), dtype=float)
    discriminator_losses = np.empty((0, 0), dtype=float)
    for epoch in range(epochs):
        start = time.time()
        
        batch_generator_losses = np.empty((0, 0), dtype=float)
        batch_discriminator_losses = np.empty((0, 0), dtype=float)
        for (batch, image_batch) in enumerate(dataset):
            scaled_batch = scale_input_images(image_batch)
            gen_loss, disc_loss = train_step(scaled_batch)
            
            if batch % 100 == 0:
                average_batch_loss =\
                   gen_loss.numpy()/int(image_batch.shape[1])
                print(f"""Epoch {epoch+1}
                        Batch {batch} Loss {average_batch_loss:.4f}""")

            batch_generator_losses = np.append(batch_generator_losses, gen_loss)
            batch_discriminator_losses = np.append(batch_discriminator_losses, disc_loss)
            
        if generator_losses.shape == (0, 0):
            generator_losses = batch_generator_losses
            discriminator_losses = batch_discriminator_losses
        else:
            generator_losses = np.vstack(
                [generator_losses, batch_generator_losses]
            )
            discriminator_losses = np.vstack(
                [discriminator_losses, batch_discriminator_losses]
            )
            
        # Saving the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

        # Producing images for the GIF
        #display.clear_output(wait=True)
        example_images = generator(seed, training=False)
        plot_examples(example_images, epoch)
        

    # Generating after the final epoch
    example_images = generator(seed, training=False)
    plot_examples(example_images, epoch)
    
    return (generator_losses, discriminator_losses)

In [ ]:
# Training the model
(generator_losses, discriminator_losses) = train(dataset, 50)